In [4]:
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
postings_1 = pd.read_csv('input/Postings_01052019_15052019.csv', sep=';')
postings_2 = pd.read_csv('input/Postings_16052019_31052019.csv', sep=';')
votes_1 = pd.read_csv('input/Votes_01052019_15052019.csv', sep=';')
votes_2 = pd.read_csv('input/Votes_16052019_31052019.csv', sep=';')
follow_ignore = pd.read_csv("input/Following_Ignoring_Relationships_01052019_31052019.csv", sep=";")